In [3]:
import sys
!{sys.executable} -m pip install sounddevice soundfile speechrecognition


In [4]:
!pip3 install sounddevice scipy SpeechRecognition ipywidgets


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
# ✅ Step 1: Imports
import os
import sys
import time
import threading

sys.path.append(os.path.abspath(".."))

from utils.voice_input import record_voice, transcribe_and_append
import ipywidgets as widgets
from IPython.display import display, clear_output

# ✅ Step 2: UI Setup
record_btn = widgets.Button(description="🎙️ Start Recording", button_style="danger")
clear_btn = widgets.Button(description="🧹 Clear Previous Transcript", button_style="warning")
duration_slider = widgets.IntSlider(
    value=30,
    min=5,
    max=120,
    step=5,
    description='⏱ Duration (sec)',
    style={'description_width': 'initial'},
    continuous_update=False
)
timer_label = widgets.Label(value="⏱ 0 sec")
output = widgets.Output()

# ✅ Step 3: Timer Thread
def update_timer(duration):
    for i in range(duration + 1):
        timer_label.value = f"⏱ {i} sec"
        time.sleep(1)

# ✅ Step 4: Button Functionalities
def start_recording(_):
    with output:
        clear_output()
        duration = duration_slider.value
        timer_thread = threading.Thread(target=update_timer, args=(duration,))
        timer_thread.start()

        wav_path, timestamp = record_voice(duration)
        text = transcribe_and_append(wav_path, timestamp)
        timer_label.value = "✅ Recording complete!"

        if text:
            print("\n🔍 Preview:\n")
            print(text[:1000])

def clear_transcript(_):
    latest_path = "../assets/voice_transcripts/transcript_latest.txt"
    if os.path.exists(latest_path):
        with open(latest_path, "w", encoding="utf-8") as f:
            f.write("")
        with output:
            clear_output()
            print("🧹 transcript_latest.txt cleared.")
    else:
        with output:
            clear_output()
            print("⚠️ transcript_latest.txt does not exist yet.")

record_btn.on_click(start_recording)
clear_btn.on_click(clear_transcript)

# ✅ Step 5: Display UI
display(widgets.VBox([duration_slider, timer_label, record_btn, clear_btn, output]))
